# NLP_3 Vocabulary and Phrase Matching with SpaCy

[Python for NLP: Vocabulary and Phrase Matching with SpaCy](https://stackabuse.com/python-for-nlp-vocabulary-and-phrase-matching-with-spacy/)

## Rule-Based Matching

### Creating Matcher Object

SpaCy 套件提供了```Matcher()```函式來進行詞組匹配，首先定義出想要匹配的 pattern，接著再把不同的 patterns 加到 Matcher 的物件當中，最在把目標文句放到該 Matcher 物件中進行匹配。

In [2]:
import spacy
nlp = spacy.load('en_core_web_sm') # 使用 SpaCy 套件都要記得先引進語言模型

from spacy.matcher import Matcher
m_tool = Matcher(nlp.vocab) # 生成 Matcher 物件

### Defining Patterns

接著說明如何定義出不同的 patterns，假設想要配對到四種 patterns，分別為 quick-brown-fox、quick brown fox、quickbrownfox 和 quick brownfox。

In [3]:
p1 = [{'LOWER': 'quickbrownfox'}]
p2 = [{'LOWER': 'quick'}, {'IS_PUNCT': True}, {'LOWER': 'brown'}, {'IS_PUNCT': True}, {'LOWER': 'fox'}]
p3 = [{'LOWER': 'quick'}, {'LOWER': 'brown'}, {'LOWER': 'fox'}]
p4 =  [{'LOWER': 'quick'}, {'LOWER': 'brownfox'}]

* p1 對應到 quickbrownfox
* p2 對應到 quick-brown-fox
* p3 對應到 quick brown fox
* p4 對應到 quick brownfox

patterns 當中的 token 性質```LOWER```代表該詞組在進行匹配之前會先轉成小寫，token 性質```IS_PUNCT```代表可以是任一標點符號。

所有的 patterns 都定義好了之後，就可以透過```add()```來把這些 patterns 加到先前生成的 Matcher 物件當中。

In [4]:
m_tool.add('QBF', None, p1, p2, p3, p4) # QBF 代表該 Matcher 的名字

### Applying Matcher to the Document

接下來要實際針對目標文句進行匹配了，首先一樣先把目標文句放入語言模型中進行處理（tokenization），處理後的結果再放入 Matcher 物件當中。

得到的結果為一個 list，裡頭包含所有被配對到的詞組，每一個詞組都透過 tuple 呈現，index 0 為該詞組所對應到的 Matcher 的 ID，這邊四個 tuple 的 index 0 都相同，是因為都是來自 QBF Matcher 匹配而生的詞組。tuple 的 index 1 代表被配對到的詞組的 starting index，index 2 則為 ending index。可以透過 for loop 清楚呈現出具體配對到了哪些字詞。

In [8]:
sentence = nlp('The quick-brown-fox jumps over the lazy dog. The quick brown fox eats well. \
               the quickbrownfox is dead. the dog misses the quick brownfox')

phrase_matches = m_tool(sentence)
print(phrase_matches)

[(12825528024649263697, 1, 6), (12825528024649263697, 13, 16), (12825528024649263697, 21, 22), (12825528024649263697, 29, 31)]


In [9]:
for match_id, start, end in phrase_matches:
    string_id = nlp.vocab.strings[match_id] # 找出 Matcher ID 所對應到的 Matcher 名稱
    span = sentence[start:end] # 利用 slicing 找出匹配出的詞         
    print(match_id, string_id, start, end, span.text)

12825528024649263697 QBF 1 6 quick-brown-fox
12825528024649263697 QBF 13 16 quick brown fox
12825528024649263697 QBF 21 22 quickbrownfox
12825528024649263697 QBF 29 31 quick brownfox


### More Options for Rule-Based Matching

SpaCy 套件有[正式的文件](https://spacy.io/usage/linguistic-features#adding-patterns-attributes)說明 phrase matching 還有哪些可以使用的性質。舉例來說，還可以透過```*```性質來匹配單一或更多的同一 token。

In [10]:
m_tool.remove('QBF') # 先用 remove() 移除先前的 QBF Matcher

p1 = [{'LOWER': 'quick'}, {'IS_PUNCT': True, 'OP':'*'}, {'LOWER': 'brown'}, \
      {'IS_PUNCT': True, 'OP':'*'}, {'LOWER': 'fox'}] # 定義 pattern，加入 'OP':'*' 說明某個 token 可多次重複或不存在
m_tool.add('QBF', None, p1)

sentence = nlp('The quick--brown--fox jumps over the  quick-brown---fox')

phrase_matches = m_tool(sentence)

for match_id, start, end in phrase_matches:
    string_id = nlp.vocab.strings[match_id]  
    span = sentence[start:end]                   
    print(match_id, string_id, start, end, span.text)

12825528024649263697 QBF 1 6 quick--brown--fox
12825528024649263697 QBF 10 15 quick-brown---fox


## Phrase-Based Matching

除了定義出 patterns 或者規則來進行詞組配對之外，另外一種更直接的方式是直接說明想要匹配的詞組，進行實作前首先爬取目標文件。

In [1]:
import requests
from bs4 import BeautifulSoup
import re

scrapped_data = requests.get('https://en.wikipedia.org/wiki/Artificial_intelligence') # 取得頁面

parsed_article = BeautifulSoup(scrapped_data.text, 'html.parser') # 解析網頁內容

paragraphs = parsed_article.find_all('p') # 把網頁主要內文找出來

article_text = ""

for p in paragraphs:
    article_text += p.text

processed_article = article_text.lower()
processed_article = re.sub('[^a-zA-Z]', ' ', processed_article) # 非字母開頭的字以空字串取代
processed_article = re.sub('\s+', ' ', processed_article) # 任何空白字符以空字串取代

### Create Phrase Matcher Object And Applying Matcher to the Document

準備好了目標文件後，同樣導入語言模型並且建立 Matcher，和建立 rule-based 的```Matcher()```不同，phrase-based 使用```PhraseMatcher()```生成 Matcher 物件。接下來直接說明想要匹配的詞組，分別為 machine learning、robots 和 intelligent agents，然而在把這些匹配目標放入 Matcher 物件之前，必須放入語言模型中處理完畢，再透過```add()```放入模型中。最後找出匹配結果的流程和 rule-based Matcher 完全相同。

In [2]:
import spacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import PhraseMatcher
phrase_matcher = PhraseMatcher(nlp.vocab)

phrases = ['machine learning', 'robots', 'intelligent agents']

patterns = [nlp(text) for text in phrases] # 

phrase_matcher.add('AI', None, *patterns) # * 代表解壓縮符號，把 list 當中每一個 element 拆成不同的參數放入

In [3]:
sentence = nlp(processed_article)

matched_phrases = phrase_matcher(sentence)

for match_id, start, end in matched_phrases:
    string_id = nlp.vocab.strings[match_id]  
    span = sentence[start:end]                   
    print(match_id, string_id, start, end, span.text)

5530044837203964789 AI 35 37 intelligent agents
5530044837203964789 AI 259 261 machine learning
5530044837203964789 AI 548 549 robots
5530044837203964789 AI 1165 1167 machine learning
5530044837203964789 AI 1480 1482 intelligent agents
5530044837203964789 AI 3134 3136 intelligent agents
5530044837203964789 AI 3293 3295 machine learning
5530044837203964789 AI 3833 3834 robots
5530044837203964789 AI 5291 5292 robots
5530044837203964789 AI 5368 5369 robots
5530044837203964789 AI 6850 6852 machine learning
5530044837203964789 AI 6862 6864 machine learning
5530044837203964789 AI 7583 7585 machine learning
5530044837203964789 AI 7722 7724 machine learning
5530044837203964789 AI 8089 8091 machine learning
5530044837203964789 AI 9621 9622 robots
5530044837203964789 AI 9686 9687 robots
5530044837203964789 AI 10138 10140 machine learning
5530044837203964789 AI 10470 10472 machine learning
5530044837203964789 AI 11627 11628 robots
5530044837203964789 AI 12104 12105 robots
5530044837203964789 AI 1

## Stop Words

SpaCy 套件亦提供了基本的停用字(stop words)，所謂停用字代表文句中一些經常出現卻沒有太特別意義的單詞，在進階的 NLP 處理當中經常會將這些字詞排除。語言模型的```Defaults.stop_words```可以呈現預設的停用字，```is_stop```可以檢視特定字詞是否為停用字，```Defaults.stop_words.add()```可以在套件中增加特定停用字，反之要移除則是```Defaults.stop_words.remove()```。若想要將套件中的特定字轉變為停用字，只要將其```is_stop```轉為 True 即可。

In [41]:
import spacy
sp = spacy.load('en_core_web_sm')
print(sp.Defaults.stop_words)

{'keep', 'six', 'therein', 'regarding', 'although', 'among', 'be', 'had', 'indeed', 'ours', 'ourselves', 'besides', 'herein', 'who', 'whoever', "'d", 'anyone', 'more', '‘re', 'across', 'is', 'other', 'thereupon', 'seemed', 'ca', 'fifty', 'down', 'those', 'hence', 'my', 'make', 'also', 'nothing', 'sometimes', 'third', 'whole', 'nor', 'via', 'once', 'may', 'whose', 'can', 'seems', 'hundred', 'after', 'formerly', "'ll", 'on', 'sometime', 'since', 'that', 'neither', 'seeming', 'under', 'beforehand', 'others', 'during', 'whence', 'back', 'wherein', 'made', '’d', 'about', 'are', 'enough', 'first', 'has', 'else', 'something', 'thence', 'she', 'one', 'eleven', 'hers', 'was', 'am', 'he', 'amount', "n't", '‘m', 'became', 'empty', "'ve", 'hereby', 'below', 'last', 'doing', 'between', 'quite', 'top', 'you', 'never', 'the', "'s", 'almost', 'often', 'becomes', 'ever', 'these', 'thus', 'whereupon', "'m", 'take', 'already', 'mostly', 'really', 'again', 'rather', 'if', 'through', 'moreover', 'which', '

In [44]:
sp.vocab['wonder'].is_stop

False

In [43]:
sp.Defaults.stop_words.add('wonder')
sp.Defaults.stop_words.remove('wonder')

In [37]:
sp.vocab['wonder'].is_stop = True